# 1. Workspace preparation

In [1]:
!pip install pygsheets

     |████████████████████████████████| 147 kB 5.0 MB/s 


In [ ]:
import pandas as pd
import pygsheets
import psycopg2

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


# 2. Open a spreadsheet to receive the cleaned data

In [ ]:
g_sheets = pygsheets.authorize(service_file='/content/drive/MyDrive/automation_dashboards/dashboard-admon-project.json')

In [ ]:
modeling = g_sheets.open('modeling_data')
w_modeling = modeling.sheet1

# 3. Connect with database

In [ ]:
conn = psycopg2.connect(
        host='ec2-3-230-199-240.compute-1.amazonaws.com',
        database='d3df54jud662it',
        user='avcobtmtntvzvv',
        password='9f675a820ce2a07d446dd03981071563fedb390ed9ccbbacf8cc3b70e7d16e0e')

## 3.1 Get tables from database

In [ ]:
cursor_cities = conn.cursor()
q_cities = '''SELECT * FROM cities ORDER BY City_id'''

cursor_countries = conn.cursor()
q_countries = '''SELECT * FROM countries'''

cursor_countries_cities = conn.cursor()
q_countries_cities = '''SELECT * FROM countries_cities ORDER BY country_id'''

cursor_users = conn.cursor()
q_users = '''SELECT *, EXTRACT (MONTH FROM created_at) AS created_month
FROM users;'''

cursor_professionals = conn.cursor()
q_professionals = '''SELECT * FROM professionals'''

cursor_sessions = conn.cursor()
q_sessions = '''SELECT *, EXTRACT (MONTH FROM date_login) AS active_month
from sessions;'''

## 3.2 Extract the column names

In [ ]:
# Cities

cursor_cities.execute(q_cities)
cities_records = cursor_cities.fetchall()

cities_col_names = []
for col_name in cursor_cities.description:
    cities_col_names.append(col_name[0])

In [ ]:
# Countries

cursor_countries.execute(q_countries)
countries_records = cursor_countries.fetchall()

countries_col_names = []
for col_name in cursor_countries.description:
    countries_col_names.append(col_name[0])

In [ ]:
# Countries_cities

cursor_countries_cities.execute(q_countries_cities)
countries_cities_records = cursor_countries_cities.fetchall()

countries_cities_col_names = []
for col_name in cursor_countries_cities.description:
    countries_cities_col_names.append(col_name[0])

In [ ]:
# Users

cursor_users.execute(q_users)
users_records = cursor_users.fetchall()

users_col_names = []
for col_name in cursor_users.description:
    users_col_names.append(col_name[0])

In [ ]:
# Professionals

cursor_professionals.execute(q_professionals)
professionals_records = cursor_professionals.fetchall()

professionals_col_names = []
for col_name in cursor_professionals.description:
    professionals_col_names.append(col_name[0])

In [ ]:
# Sessions

cursor_sessions.execute(q_sessions)
sessions_records = cursor_sessions.fetchall()

sessions_col_names = []
for col_name in cursor_sessions.description:
    sessions_col_names.append(col_name[0])

# 4. Become in Dataframes

In [ ]:
# Countries
df_countries = pd.DataFrame(countries_records, columns=countries_col_names)
df_countries

In [ ]:
df_countries.rename(columns={'name': 'country_name'}, inplace=True)
df_countries.drop(['created_at','created_by','updated_at','updated_by'], 1, inplace=True)

In [ ]:
#Cities
df_cities = pd.DataFrame(cities_records, columns=cities_col_names)
df_cities

In [ ]:
df_cities.rename(columns={'name': 'city_name'}, inplace=True)
df_cities.drop(['created_at','created_by','updated_at','updated_by'], 1, inplace=True)

In [ ]:
# Countries and Cities
df_countries_cities = pd.DataFrame(countries_cities_records, columns=countries_cities_col_names)
df_countries_cities

In [ ]:
# Users
df_users = pd.DataFrame(users_records, columns=users_col_names)
df_users

In [ ]:
df_users.rename(columns={'name': 'user_name', 'created_at': 'user_created_at'}, inplace=True)
df_users.drop(['email','password','profile_picture','created_by','updated_at','updated_by'], 1, inplace=True)

In [ ]:
#Professionals
df_professionals = pd.DataFrame(professionals_records, columns=professionals_col_names)
df_professionals

In [ ]:
df_professionals.drop(['phone_number','subdomain','description','created_at','created_by','updated_at','updated_by'], 1, inplace=True)

In [ ]:
#Sessions
df_sessions = pd.DataFrame(sessions_records, columns=sessions_col_names)
df_sessions

# 5. Merge all dataframes

Cities and Countries

In [ ]:
df_countries_pivote = pd.merge(df_countries, df_countries_cities, on='country_id')

In [ ]:
df_cities_and_countries = pd.merge(df_countries_pivote, df_cities, on='city_id')

Professionals and users

In [ ]:
df_professionals['user_id'] = df_professionals['professional_id']

In [ ]:
# # to change use .astype() 
# df_professionals['user_id'] = df_professionals.user_id.astype(int)

In [ ]:
df_professionals_users = pd.merge(df_professionals, df_users, on='user_id')

Professional_users and countries

In [ ]:
df_countries['country_id'] = df_countries.country_id.astype(int)

In [ ]:
df_professionals_countries = pd.merge(df_professionals_users, df_countries, on='country_id')
df_professionals_countries

,professional_id,first_name,last_name,birth_date,gender,english_level,looking_job,location_preference,user_id,user_name,active,user_created_at,role_id,created_month,country_id,country_name


Data Modelin

In [ ]:
df_modeling = pd.merge(df_sessions, df_professionals_countries, on='user_id')
df_modeling

,session_id,ip_address,devices,OS,browser,date_login,date_logout,active_month,professional_id,first_name,last_name,birth_date,gender,english_level,looking_job,location_preference,user_id,user_name,active,user_created_at,role_id,created_month,country_id,country_name



# 6. Send data to a spreadsheets

In [ ]:
w_modeling.set_dataframe(df_modeling,(1,1))